In [2]:
df = pd.read_csv("input/train.csv", index_col=False)

In [3]:
df.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
0,1,2014-01-29,69,38.0,7.0,10.0,1.0,2001.0,2.0,11.0,...,1,0,0,2,8,1,0,19,2,9500000
1,2,2014-04-07,55,NaN,2.0,1.0,4.0,NaN,2.0,10.0,...,0,0,0,0,4,0,0,2,0,3837949
2,3,2012-05-18,30,16.0,2.0,NaN,NaN,NaN,NaN,NaN,...,46,9,2,11,38,1,8,97,11,6250000
3,4,2013-02-08,44,43.0,1.0,NaN,NaN,NaN,NaN,NaN,...,17,4,1,12,12,0,1,55,7,2000000
4,5,2014-01-10,45,28.0,3.0,5.0,2.0,1960.0,2.0,5.0,...,20,2,0,4,16,1,4,47,5,6700000


In [4]:
df.shape

(24376, 292)

# Preprocessing

## Check types

In [5]:
df.dtypes.value_counts()

int64      157
float64    119
object      16
dtype: int64

## Get notnums w/ indices

In [6]:
notnum = df.dtypes[df.dtypes == type(object)]
notnum

f1      object
f11     object
f12     object
f29     object
f33     object
f34     object
f35     object
f36     object
f37     object
f38     object
f39     object
f40     object
f106    object
f114    object
f118    object
f152    object
dtype: object

In [7]:
notnum_index = list(notnum.index)
print(notnum_index)

['f1', 'f11', 'f12', 'f29', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f106', 'f114', 'f118', 'f152']


In [8]:
df[notnum_index].head()

,f1,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014-01-29,Investment,Juzhnoe Butovo,no,no,no,no,no,no,no,no,no,no,no,no,good
1,2014-04-07,OwnerOccupier,Poselenie Filimonkovskoe,no,no,no,no,no,no,no,no,no,no,no,no,no data
2,2012-05-18,Investment,Lomonosovskoe,no,no,no,no,no,no,no,no,no,no,no,no,satisfactory
3,2013-02-08,Investment,Juzhnoe Tushino,no,no,no,no,no,no,no,no,no,no,no,no,poor
4,2014-01-10,Investment,Ochakovo-Matveevskoe,no,yes,no,no,no,no,no,no,no,no,no,no,satisfactory


## Process f1

In [9]:
ymd = df['f1'].str.split('-', expand=True).astype('int64').rename(columns={0: 'f1y', 1: 'f1m', 2: 'f1d'})
ymd

,f1y,f1m,f1d
0,2014,1,29
1,2014,4,7
2,2012,5,18
3,2013,2,8
4,2014,1,10
...,...,...,...
24371,2014,12,21
24372,2012,5,10
24373,2013,12,18
24374,2014,12,8


In [10]:
df_f1 = df.drop(['f1'], axis=1)
df_f1.insert(1, 'f1y', ymd['f1y'])
df_f1.insert(2, 'f1m', ymd['f1m'])
df_f1.insert(3, 'f1d', ymd['f1d'])

In [11]:
notnum_index = ['f1y', 'f1m', 'f1d'] + notnum_index[1:]
print(notnum_index)

['f1y', 'f1m', 'f1d', 'f11', 'f12', 'f29', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f106', 'f114', 'f118', 'f152']


In [12]:
df_f1[notnum_index].head()

,f1y,f1m,f1d,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014,1,29,Investment,Juzhnoe Butovo,no,no,no,no,no,no,no,no,no,no,no,no,good
1,2014,4,7,OwnerOccupier,Poselenie Filimonkovskoe,no,no,no,no,no,no,no,no,no,no,no,no,no data
2,2012,5,18,Investment,Lomonosovskoe,no,no,no,no,no,no,no,no,no,no,no,no,satisfactory
3,2013,2,8,Investment,Juzhnoe Tushino,no,no,no,no,no,no,no,no,no,no,no,no,poor
4,2014,1,10,Investment,Ochakovo-Matveevskoe,no,yes,no,no,no,no,no,no,no,no,no,no,satisfactory


## Process f11

In [13]:
f11_unique = df_f1['f11'].unique()
f11_unique

array(['Investment', 'OwnerOccupier'], dtype=object)

In [14]:
f11_encode = dict(zip(f11_unique, range(len(f11_unique))))
f11_encode

{'Investment': 0, 'OwnerOccupier': 1}

In [15]:
df_f11 = df_f1.replace({'f11': f11_encode})
df_f11[notnum_index].head()

,f1y,f1m,f1d,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014,1,29,0,Juzhnoe Butovo,no,no,no,no,no,no,no,no,no,no,no,no,good
1,2014,4,7,1,Poselenie Filimonkovskoe,no,no,no,no,no,no,no,no,no,no,no,no,no data
2,2012,5,18,0,Lomonosovskoe,no,no,no,no,no,no,no,no,no,no,no,no,satisfactory
3,2013,2,8,0,Juzhnoe Tushino,no,no,no,no,no,no,no,no,no,no,no,no,poor
4,2014,1,10,0,Ochakovo-Matveevskoe,no,yes,no,no,no,no,no,no,no,no,no,no,satisfactory


## Process f12

In [16]:
f12_unique = df_f11['f12'].unique()
f12_unique

array(['Juzhnoe Butovo', 'Poselenie Filimonkovskoe', 'Lomonosovskoe',
       'Juzhnoe Tushino', 'Ochakovo-Matveevskoe', 'Poselenie Sosenskoe',
       'Orehovo-Borisovo Juzhnoe', 'Hovrino', 'Sokol',
       'Nagatinskij Zaton', "Kon'kovo", 'Obruchevskoe',
       'Poselenie Vnukovskoe', 'Preobrazhenskoe', 'Poselenie Moskovskij',
       'Bibirevo', 'Jasenevo', 'Mitino', 'Birjulevo Vostochnoe',
       'Koptevo', 'Beskudnikovskoe', 'Butyrskoe', 'Losinoostrovskoe',
       'Novo-Peredelkino', 'Caricyno', 'Veshnjaki', 'Tverskoe', 'Perovo',
       'Jaroslavskoe', 'Strogino', 'Nekrasovka', 'Horoshevo-Mnevniki',
       'Filevskij Park', 'Bogorodskoe', 'Savelki',
       'Poselenie Desjonovskoe', "Chertanovo Central'noe",
       'Timirjazevskoe', 'Zapadnoe Degunino', 'Pokrovskoe Streshnevo',
       'Severnoe Butovo', 'Kapotnja', 'Sviblovo', 'Danilovskoe',
       'Sokolinaja Gora', 'Vostochnoe Izmajlovo', 'Matushkino',
       'Krjukovo', "Krasnosel'skoe", 'Ajeroport', 'Taganskoe',
       'Cheremushki

In [17]:
f12_encode = dict(zip(f12_unique, range(len(f12_unique))))
f12_encode

{'Juzhnoe Butovo': 0,
 'Poselenie Filimonkovskoe': 1,
 'Lomonosovskoe': 2,
 'Juzhnoe Tushino': 3,
 'Ochakovo-Matveevskoe': 4,
 'Poselenie Sosenskoe': 5,
 'Orehovo-Borisovo Juzhnoe': 6,
 'Hovrino': 7,
 'Sokol': 8,
 'Nagatinskij Zaton': 9,
 "Kon'kovo": 10,
 'Obruchevskoe': 11,
 'Poselenie Vnukovskoe': 12,
 'Preobrazhenskoe': 13,
 'Poselenie Moskovskij': 14,
 'Bibirevo': 15,
 'Jasenevo': 16,
 'Mitino': 17,
 'Birjulevo Vostochnoe': 18,
 'Koptevo': 19,
 'Beskudnikovskoe': 20,
 'Butyrskoe': 21,
 'Losinoostrovskoe': 22,
 'Novo-Peredelkino': 23,
 'Caricyno': 24,
 'Veshnjaki': 25,
 'Tverskoe': 26,
 'Perovo': 27,
 'Jaroslavskoe': 28,
 'Strogino': 29,
 'Nekrasovka': 30,
 'Horoshevo-Mnevniki': 31,
 'Filevskij Park': 32,
 'Bogorodskoe': 33,
 'Savelki': 34,
 'Poselenie Desjonovskoe': 35,
 "Chertanovo Central'noe": 36,
 'Timirjazevskoe': 37,
 'Zapadnoe Degunino': 38,
 'Pokrovskoe Streshnevo': 39,
 'Severnoe Butovo': 40,
 'Kapotnja': 41,
 'Sviblovo': 42,
 'Danilovskoe': 43,
 'Sokolinaja Gora': 44,
 'V

In [18]:
df_f12 = df_f11.replace({'f12': f12_encode})
df_f12[notnum_index].head()

,f1y,f1m,f1d,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014,1,29,0,0,no,no,no,no,no,no,no,no,no,no,no,no,good
1,2014,4,7,1,1,no,no,no,no,no,no,no,no,no,no,no,no,no data
2,2012,5,18,0,2,no,no,no,no,no,no,no,no,no,no,no,no,satisfactory
3,2013,2,8,0,3,no,no,no,no,no,no,no,no,no,no,no,no,poor
4,2014,1,10,0,4,no,yes,no,no,no,no,no,no,no,no,no,no,satisfactory


## Process f29 - f118

In [19]:
notnum_bools = notnum_index[5:-1]
print(notnum_bools)

['f29', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f106', 'f114', 'f118']


In [20]:
bools_unique = set(np.array([df_f12[col].unique() for col in notnum_bools]).flatten())
bools_unique

{'no', 'yes'}

In [21]:
bools_encode = dict(zip(bools_unique, range(len(bools_unique))))
bools_encode

{'yes': 0, 'no': 1}

In [22]:
df_bools = df_f12.replace(dict.fromkeys(notnum_bools, bools_encode))
df_bools[notnum_index].head()

,f1y,f1m,f1d,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014,1,29,0,0,1,1,1,1,1,1,1,1,1,1,1,1,good
1,2014,4,7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,no data
2,2012,5,18,0,2,1,1,1,1,1,1,1,1,1,1,1,1,satisfactory
3,2013,2,8,0,3,1,1,1,1,1,1,1,1,1,1,1,1,poor
4,2014,1,10,0,4,1,0,1,1,1,1,1,1,1,1,1,1,satisfactory


## Process f152

In [23]:
f152_unique = df_bools['f152'].unique()
f152_unique

array(['good', 'no data', 'satisfactory', 'poor', 'excellent'],
      dtype=object)

In [24]:
f152_encode = dict(zip(['no data', 'poor', 'satisfactory', 'good', 'excellent'], range(5)))
f152_encode

{'no data': 0, 'poor': 1, 'satisfactory': 2, 'good': 3, 'excellent': 4}

In [25]:
df_f152 = df_bools.replace({'f152': f152_encode})
df_f152[notnum_index].head()

,f1y,f1m,f1d,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014,1,29,0,0,1,1,1,1,1,1,1,1,1,1,1,1,3
1,2014,4,7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
2,2012,5,18,0,2,1,1,1,1,1,1,1,1,1,1,1,1,2
3,2013,2,8,0,3,1,1,1,1,1,1,1,1,1,1,1,1,1
4,2014,1,10,0,4,1,0,1,1,1,1,1,1,1,1,1,1,2


## Obtain result

In [26]:
df_final = df_f152.copy()
df_final.head()

,id,f1y,f1m,f1d,f2,f3,f4,f5,f6,f7,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
0,1,2014,1,29,69,38.0,7.0,10.0,1.0,2001.0,...,1,0,0,2,8,1,0,19,2,9500000
1,2,2014,4,7,55,NaN,2.0,1.0,4.0,NaN,...,0,0,0,0,4,0,0,2,0,3837949
2,3,2012,5,18,30,16.0,2.0,NaN,NaN,NaN,...,46,9,2,11,38,1,8,97,11,6250000
3,4,2013,2,8,44,43.0,1.0,NaN,NaN,NaN,...,17,4,1,12,12,0,1,55,7,2000000
4,5,2014,1,10,45,28.0,3.0,5.0,2.0,1960.0,...,20,2,0,4,16,1,4,47,5,6700000


## Check NaNs

In [27]:
nans_cols = df_final.isna().sum()
nans_cols[nans_cols > 0].count()

51

In [28]:
df_final[nans_cols[nans_cols > 0].index].dtypes.unique()

array([dtype('float64')], dtype=object)

In [29]:
nans_rows = df_final.isna().sum(axis=1)
nans_rows[nans_rows >= 40]

172      40
554      40
785      40
1052     40
1359     40
         ..
22921    40
23466    40
23550    40
24236    40
24316    41
Length: 155, dtype: int64

In [30]:
print(list(nans_rows[nans_rows >= 40].index))

[172, 554, 785, 1052, 1359, 1495, 1646, 1812, 1997, 2133, 2249, 2302, 2529, 2542, 2678, 2684, 2781, 2801, 2861, 3251, 3254, 3699, 3904, 3959, 4208, 4365, 4544, 4656, 4834, 4929, 4941, 5151, 5159, 5277, 5474, 5621, 5634, 5691, 5805, 6092, 6191, 6256, 6347, 6388, 6524, 6862, 6876, 7066, 7202, 7259, 7506, 7508, 7575, 7618, 7625, 7694, 7796, 7980, 8119, 8184, 8484, 8673, 8834, 9290, 9632, 9693, 9786, 10247, 10427, 10572, 10639, 11016, 11281, 11394, 11648, 11857, 12249, 12556, 12625, 13322, 13330, 13498, 13592, 13696, 13772, 13911, 13996, 14014, 14062, 14331, 14345, 14518, 14925, 14999, 15026, 15086, 15313, 15359, 15493, 15679, 15812, 15916, 16036, 16129, 16532, 16820, 16965, 17116, 17534, 17574, 17971, 18011, 18037, 18053, 18256, 18311, 18334, 18429, 18475, 18532, 18630, 18632, 18691, 19155, 19187, 19253, 19429, 19523, 19792, 20051, 20087, 20279, 20435, 20650, 20927, 21346, 21410, 21508, 21570, 21735, 22029, 22096, 22195, 22210, 22235, 22258, 22324, 22356, 22477, 22608, 22921, 23466, 23550

## Remove too dirty entries

In [31]:
df_nans = df_final.drop(list(nans_rows[nans_rows >= 40].index))
df_nans.shape

(24221, 294)

## Impute missing values with KNN

In [32]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(weights='distance')

In [33]:
df_imp = imputer.fit_transform(df_nans)

In [34]:
df_imp2 = pd.DataFrame(df_imp, columns=df_final.columns)
df_imp2.head()

,id,f1y,f1m,f1d,f2,f3,f4,f5,f6,f7,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
0,1.0,2014.0,1.0,29.0,69.0,38.00000,7.0,10.00000,1.000000,2001.000000,...,1.0,0.0,0.0,2.0,8.0,1.0,0.0,19.0,2.0,9500000.0
1,2.0,2014.0,4.0,7.0,55.0,48.13174,2.0,1.00000,4.000000,1591.150833,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,3837949.0
2,3.0,2012.0,5.0,18.0,30.0,16.00000,2.0,7.29370,3.732457,1961.927529,...,46.0,9.0,2.0,11.0,38.0,1.0,8.0,97.0,11.0,6250000.0
3,4.0,2013.0,2.0,8.0,44.0,43.00000,1.0,8.78533,1.428214,1970.661667,...,17.0,4.0,1.0,12.0,12.0,0.0,1.0,55.0,7.0,2000000.0
4,5.0,2014.0,1.0,10.0,45.0,28.00000,3.0,5.00000,2.000000,1960.000000,...,20.0,2.0,0.0,4.0,16.0,1.0,4.0,47.0,5.0,6700000.0


In [35]:
imputed = nans_cols[nans_cols > 0]
df_full = df_nans.copy()
df_full.reset_index(drop=True, inplace=True)
df_full.head()

,id,f1y,f1m,f1d,f2,f3,f4,f5,f6,f7,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
0,1,2014,1,29,69,38.0,7.0,10.0,1.0,2001.0,...,1,0,0,2,8,1,0,19,2,9500000
1,2,2014,4,7,55,NaN,2.0,1.0,4.0,NaN,...,0,0,0,0,4,0,0,2,0,3837949
2,3,2012,5,18,30,16.0,2.0,NaN,NaN,NaN,...,46,9,2,11,38,1,8,97,11,6250000
3,4,2013,2,8,44,43.0,1.0,NaN,NaN,NaN,...,17,4,1,12,12,0,1,55,7,2000000
4,5,2014,1,10,45,28.0,3.0,5.0,2.0,1960.0,...,20,2,0,4,16,1,4,47,5,6700000


In [36]:
df_full[-3:]

,id,f1y,f1m,f1d,f2,f3,f4,f5,f6,f7,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
24218,24374,2013,12,18,85,NaN,16.0,17.0,1.0,1.0,...,3,1,0,3,9,0,1,10,2,9609180
24219,24375,2014,12,8,47,27.0,2.0,10.0,2.0,1958.0,...,217,86,11,118,176,1,64,146,13,12900000
24220,24376,2015,4,30,85,60.0,10.0,22.0,4.0,2015.0,...,1,1,0,2,12,0,0,9,0,10371441


In [37]:
df_full[imputed.index] = df_imp2[imputed.index]
df_full.head()

,id,f1y,f1m,f1d,f2,f3,f4,f5,f6,f7,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
0,1,2014,1,29,69,38.00000,7.0,10.00000,1.000000,2001.000000,...,1,0,0,2,8,1,0,19,2,9500000
1,2,2014,4,7,55,48.13174,2.0,1.00000,4.000000,1591.150833,...,0,0,0,0,4,0,0,2,0,3837949
2,3,2012,5,18,30,16.00000,2.0,7.29370,3.732457,1961.927529,...,46,9,2,11,38,1,8,97,11,6250000
3,4,2013,2,8,44,43.00000,1.0,8.78533,1.428214,1970.661667,...,17,4,1,12,12,0,1,55,7,2000000
4,5,2014,1,10,45,28.00000,3.0,5.00000,2.000000,1960.000000,...,20,2,0,4,16,1,4,47,5,6700000


In [38]:
True in df_full.isna().values

False

# Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
np.random.seed(3927)

# Input

In [3]:
df_train = pd.read_csv("input/train.csv", index_col=False)
df_test = pd.read_csv("input/test.csv", index_col=False)

# Preprocess

In [4]:
def process_f1(df, index):
    ymd = df['f1'].str.split('-', expand=True).astype('int64').rename(columns={0: 'f1y', 1: 'f1m', 2: 'f1d'})

    df_res = df.drop(['f1'], axis=1)
    df_res.insert(1, 'f1y', ymd['f1y'])
    df_res.insert(2, 'f1m', ymd['f1m'])
    df_res.insert(3, 'f1d', ymd['f1d'])
    return df_res, ['f1y', 'f1m', 'f1d'] + index[1:]

In [5]:
def process_f11(df, index):
    f11_unique = df['f11'].unique()
    f11_encode = dict(zip(f11_unique, range(len(f11_unique))))
    return df.replace({'f11': f11_encode}), index

In [6]:
def process_f12(df, index):
    f12_unique = df['f12'].unique()
    f12_encode = dict(zip(f12_unique, range(len(f12_unique))))
    return df.replace({'f12': f12_encode}), index

In [7]:
def process_bool(df, index):
    bool_index = index[5:-1]
    bool_encode = {'no': 0, 'yes': 1}
    return df.replace(dict.fromkeys(bool_index, bool_encode)), index

In [8]:
def process_f152(df, index):
    f152_encode = dict(zip(['no data', 'poor', 'satisfactory', 'good', 'excellent'], range(5)))
    return df.replace({'f152': f152_encode}), index

In [9]:
def process_categorical(df):
    cat = df.dtypes[df.dtypes == type(object)]
    cat_index = list(cat.index)
    
    df_res, cat_index = process_f1(df, cat_index)
    df_res, cat_index = process_f11(df_res, cat_index)
    df_res, cat_index = process_f12(df_res, cat_index)
    df_res, cat_index = process_bool(df_res, cat_index)
    df_res, cat_index = process_f152(df_res, cat_index)
    return df_res

In [10]:
def drop_na(df):
    na_rows = df.isna().sum(axis=1)
    return df.drop(list(na_rows[na_rows >= 40].index))

In [11]:
from sklearn.impute import KNNImputer

def impute_na(df):
    imputer = KNNImputer(n_neighbors=10, weights='distance')
    imp = imputer.fit_transform(df)
    df_imp = pd.DataFrame(imp, columns=df.columns)
    
    na_cols = df.isna().sum()
    imp_cols = na_cols[na_cols > 0]

    df_res = df.copy()
    df_res.reset_index(drop=True, inplace=True)
    df_res[imp_cols.index] = df_imp[imp_cols.index]
    return df_res

In [12]:
def process_na(df, drop, impute):
    df_res = df.copy()
    if drop:
        df_res = drop_na(df_res)
    if impute:
        df_res = impute_na(df_res)
    return df_res

In [13]:
def preprocess(df, drop_na=False, impute_na=True):
    df_res = process_categorical(df)
    df_res = process_na(df_res, drop_na, impute_na)
    return df_res

In [14]:
df_train_res = preprocess(df_train, drop_na=True)
df_test_res = preprocess(df_test)

In [15]:
df_train_res.head()

,id,f1y,f1m,f1d,f2,f3,f4,f5,f6,f7,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
0,1,2014,1,29,69,38.000000,7.0,10.000000,1.000000,2001.000000,...,1,0,0,2,8,1,0,19,2,9500000
1,2,2014,4,7,55,48.693296,2.0,1.000000,4.000000,1595.848882,...,0,0,0,0,4,0,0,2,0,3837949
2,3,2012,5,18,30,16.000000,2.0,8.222948,3.431211,1963.013174,...,46,9,2,11,38,1,8,97,11,6250000
3,4,2013,2,8,44,43.000000,1.0,9.354003,1.278251,1972.770781,...,17,4,1,12,12,0,1,55,7,2000000
4,5,2014,1,10,45,28.000000,3.0,5.000000,2.000000,1960.000000,...,20,2,0,4,16,1,4,47,5,6700000


In [16]:
True in df_train_res.isna().values

False

In [17]:
df_test_res.head()

,id,f1y,f1m,f1d,f2,f3,f4,f5,f6,f7,...,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290
0,1,2014,3,29,63,63.000000,11.0,17.000000,1.000000,1467.849264,...,5,1,1,0,2,12,0,1,10,0
1,2,2013,2,19,64,45.099926,2.0,15.261046,1.383073,259.310175,...,5,1,1,0,2,12,0,0,9,0
2,3,2013,9,18,40,38.852806,4.0,17.000000,1.000000,1637.107289,...,636,371,141,26,150,249,2,105,203,13
3,4,2014,7,4,37,1.000000,25.0,1.000000,1.000000,1.000000,...,28,17,6,2,4,16,0,0,47,3
4,5,2014,9,24,127,58.000000,11.0,20.000000,1.000000,2006.000000,...,95,37,5,1,5,33,1,6,85,5


In [18]:
True in df_test_res.isna().values

False

# Prepare train-test sets

In [19]:
X_train = df_train_res.drop(['id', 'target'], axis=1)
y_train = df_train_res['target']
X_test = df_test_res.drop('id', axis=1)

In [20]:
X_train.shape, y_train.shape, X_test.shape

((24221, 292), (24221,), (6095, 292))

# Train pipeline

In [21]:
from sklearn.preprocessing import StandardScaler

def normalize(data):
    normalizer = StandardScaler()
    return pd.DataFrame(normalizer.fit_transform(data), columns=data.columns)

In [22]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

def rmsle(y, y_pred, **kwargs):
    # Allow negative predictions 
    # by incorporating prediction sign
    terms = [np.sign(pred) * np.log(1 + np.sign(pred) * pred) - np.log(1 + targ) for pred, targ in zip(y_pred, y)]
    return np.sqrt(np.sum(np.power(terms, 2.0)) / len(y))

rmsle_scorer = make_scorer(rmsle, greater_is_better=False)

def score(model, X, y, cv):
    return cross_val_score(model, X, y, cv=cv, scoring=rmsle_scorer)

In [23]:
from sklearn.model_selection import RandomizedSearchCV

def fit(model, params, X, y, n_iter, cv):
    rscv = RandomizedSearchCV(model, params, n_iter=n_iter, scoring=rmsle_scorer, refit=True, cv=cv, random_state=3927)
    rscv.fit(X, y)
    return rscv.best_estimator_, rscv.best_params_

In [24]:
def train(model, params, X, y, n_iter=10, cv=5):
    X_tr = normalize(X)
    
    print(f"Base CV scores: {score(model, X_tr, y, cv)}")

    best_model, best_params = fit(model, params, X_tr, y, n_iter, cv)
    
    print(f"Best params: {best_params}")
    print(f"Best CV scores: {score(best_model, X_tr, y, cv)}")

    return best_model

# Test pipeline

In [25]:
def test(model, X):
    X_tr = normalize(X)

    pred = model.predict(X_tr)
    data = list(zip(range(1, len(pred) + 1), pred))
    return pd.DataFrame(data, columns=['id', 'prediction'])

# Setup model

In [33]:
from sklearn.linear_model import Ridge
from scipy.stats import loguniform

model = Ridge()
params = {'alpha': loguniform(1, 100000)}

In [40]:
from sklearn.linear_model import ElasticNet
from scipy.stats import loguniform, uniform

# Optimal params for averaging scores:
# alpha = 14, 
# l1_ratio = 0.6
model = ElasticNet(max_iter=1000, random_state=3927, selection='random')
params = {'alpha': uniform(10, 20), 'l1_ratio': uniform(0.5, 0.2)}

In [48]:
from sklearn.linear_model import SGDRegressor
from scipy.stats import loguniform, uniform, norm

model = SGDRegressor(max_iter=5000, tol=10e-6, penalty='elasticnet', random_state=3927)
params = {'alpha': norm(15, 1), 'l1_ratio': uniform(0.3, 0.5)}

# Train

In [87]:
# X_train_tr = normalize(X_train)
# (X_train.max() - X_train.min()).head(30)

# print(f"Base CV scores: {score(model, X_train_tr, y_train, 5)}")

9.582172084345775

In [49]:
model = train(model, params, X_train, y_train, n_iter=10, cv=5)
model

Base CV scores: [-27.74799358 -26.26788865 -27.5260288  -25.61183766 -25.4131794 ]
Best params: {'alpha': 14.399192032033296, 'l1_ratio': 0.7180000248027849}
Best CV scores: [-0.55171556 -0.55152491 -0.58663431 -0.56147781 -0.5738154 ]


SGDRegressor(alpha=14.399192032033296, l1_ratio=0.7180000248027849,
             max_iter=5000, penalty='elasticnet', random_state=3927, tol=1e-05)

# Test

In [50]:
df_pred = test(model, X_test)
df_pred.head()

,id,prediction
0,1,6.892885e+06
1,2,6.243808e+06
2,3,9.180907e+06
3,4,7.221767e+06
4,5,9.870909e+06


# Validate

In [51]:
df_pred['prediction'][df_pred['prediction'] <= 100000]

Series([], Name: prediction, dtype: float64)

# Export

In [52]:
df_pred.to_csv("output/submission.csv", index=False)